In [2]:
import pandas as pd
import subprocess
import os
import shutil

os.chdir('/work/GNNIS')
print("Current Directory:", os.getcwd())

df = pd.read_csv('selected_smiles_solv_pka1.csv')

for index, row in df.iterrows():
    inp_smiles = row['smiles']
    #pKa_EXP = row['exp_pka']
    #dielectric_value = row['dielec']

    num_conf: int = 10
    E_avg_proton: float=-275.755
    pKa_EXP: float= 24.88
    Solvent: str="DMSO"
    dielectric_value = 46.826
    
    
    output_dir = row['id']
    os.makedirs(output_dir, exist_ok=True)
    
    temp_output_notebook_1 = 'full_pipeline_pka_gnnis-1_output.ipynb'
    final_output_notebook_1 = os.path.join(output_dir, temp_output_notebook_1)
    
    temp_output_notebook_2 = 'full_pipeline_pka_gnnis-2_output.ipynb'
    final_output_notebook_2 = os.path.join(output_dir, temp_output_notebook_2)
    
    try:
        subprocess.run(
            f"source /opt/conda/etc/profile.d/conda.sh && conda activate GNNImplicitSolvent && python -m ipykernel install --user --name GNNImplicitSolvent && "
            f"papermill full_pipeline_pka_gnnis-1.ipynb {temp_output_notebook_1} "
            f"--log-output -p inp_smiles '{inp_smiles}' -p output_dir '{output_dir}' "
            f"-p Solvent '{Solvent}' -p num_conf {num_conf} -k gnnimplicitsolvent",
            shell=True,
            check=True,
            executable="/bin/bash"
        )
        shutil.move(temp_output_notebook_1, final_output_notebook_1)
        
        subprocess.run(
            f"source /opt/conda/etc/profile.d/conda.sh && conda activate base && "
            f"papermill full_pipeline_pka_gnnis-2_v2.ipynb {temp_output_notebook_2} "
            f"--log-output -p inp_smiles '{inp_smiles}' -p output_dir '{output_dir}' "
            f"-p pKa_EXP {pKa_EXP} -p E_avg_proton {E_avg_proton} "
            f"-p num_conf {num_conf} -p dielectric_value {dielectric_value} -k python3",
            shell=True,
            check=True,
            executable="/bin/bash"
        )
        shutil.move(temp_output_notebook_2, final_output_notebook_2)
        
    except subprocess.CalledProcessError as e:
        print(f"❌ Error processing row {index}: {e}")

Current Directory: /work/GNNIS
Installed kernelspec GNNImplicitSolvent in /root/.local/share/jupyter/kernels/gnnimplicitsolvent


Input Notebook:  full_pipeline_pka_gnnis-1.ipynb
Output Notebook: full_pipeline_pka_gnnis-1_output.ipynb
Passed unknown parameter: inp_smiles
Passed unknown parameter: output_dir
Passed unknown parameter: Solvent
Passed unknown parameter: num_conf
Input notebook does not contain a cell with tag 'parameters'
Executing notebook with kernel: gnnimplicitsolvent
Executing Cell 1---------------------------------------
Ending Cell 1------------------------------------------
Executing Cell 2---------------------------------------
Ending Cell 2------------------------------------------
Executing Cell 3---------------------------------------
Ending Cell 3------------------------------------------
Executing Cell 4---------------------------------------
Ending Cell 4------------------------------------------
Executing Cell 5---------------------------------------
Ending Cell 5------------------------------------------
Executing Cell 6---------------------------------------
Ending Cell 6-----------

rep_of_cluster_2.sdf classified as deprotonate_basic
Classification for rep_of_cluster_2.sdf: deprotonate_basic
Deprotonating N-H site at atom index 3 with Mulliken charge -0.454648
Deprotonated molecule saved as '/work/M2/cluster_rep_conformers/treated/rep_of_cluster_2_deprotonated.sdf'
rep_of_cluster_1.sdf classified as deprotonate_basic
Classification for rep_of_cluster_1.sdf: deprotonate_basic
Deprotonating N-H site at atom index 1 with Mulliken charge -0.452880
Deprotonated molecule saved as '/work/M2/cluster_rep_conformers/treated/rep_of_cluster_1_deprotonated.sdf'


rep_of_cluster_2.sdf classified as deprotonate_basic
Classification for rep_of_cluster_2.sdf: deprotonate_basic
Deprotonating N-H site at atom index 3 with Mulliken charge -0.454648
Deprotonated molecule saved as '/work/M2/cluster_rep_conformers/treated/rep_of_cluster_2_deprotonated.sdf'
rep_of_cluster_1.sdf classified as deprotonate_basic
Classification for rep_of_cluster_1.sdf: deprotonate_basic
Deprotonating N-H site at atom index 1 with Mulliken charge -0.452880
Deprotonated molecule saved as '/work/M2/cluster_rep_conformers/treated/rep_of_cluster_1_deprotonated.sdf'

Ending Cell 45-----------------------------------------
Executing Cell 46--------------------------------------
Found 2 SDF files in cluster_rep_conformers
Basis sets: ['aug-cc-pVDZ']
XC functionals: ['M06-2X']
Dielectric constant: 46.826
Charge of cluster_rep_conformers/rep_of_cluster_1_deprotonated.sdf: -1
#INFO: **** input file is /work/dft_main.py ****
from rdkit import Chem
from pyscf import gto, dft
from gpu4pys